In [ ]:
# Parse data from MHC Excel files

import polars as pl
import polars.selectors as cs

# Configuration
prefix = '/master/abagwell/workspace/MHC/'
# For applying parental information for haplotype inference
demographics = '/master/abagwell/variant-analysis/resources/rhesus/pedigree/demographics.tsv'  # Last updated 2025-12-11
# For applying manual corrections to haplotyping
modifications_file = '/master/abagwell/workspace/MHC/modifications.diplotype.tsv'
# For updating batch names to simpler, standardized names.
# First column should be called "Old" with the original names.
# And the second column called "New" with the updated names.
batch_map = '/master/abagwell/workspace/MHC/batch_map.tsv'

# Read in list of input files
# File contains a columns for batch names and Excel file names
#input_files = pl.read_csv(prefix + 'input_files.tsv', separator='\t', comment_prefix="#")
input_files = pl.read_csv(prefix + 'input_files.combined.tsv', separator='\t', comment_prefix="#")

# Values to treat as nulls
null_values = ["", " ", "?", "A-unk", "DPA-unk", "DPB-unk", "DQA-unk", "DQA_unk", "DQB?"]


# Parse the "Abbreviated Haplotypes" sheet from each Excel file
def parse_excel(file):
    return pl.read_excel(f"{prefix}input/{file}", sheet_name='Abbreviated Haplotypes',
        #engine='xlsx2csv', engine_options={'null_values': null_values}
        ).rename({
            # Rename columns to make different columns names across files match
            'Client ID': 'Animal ID',
            'client_id': 'Animal ID',
            'OC ID': 'GS ID',
            'File Source': 'Batch',
            'Mamu-A Haplotype 1': 'MHC-A Haplotype 1',
            'Mamu-A Haplotype 2': 'MHC-A Haplotype 2',
            'Mamu-B Haplotype 1': 'MHC-B Haplotype 1',
            'Mamu-B Haplotype 2': 'MHC-B Haplotype 2',
            'Mamu-DRB Haplotype 1': 'MHC-DRB Haplotype 1',
            'Mamu-DRB Haplotype 2': 'MHC-DRB Haplotype 2',
            'Mamu-DQA Haplotype 1': 'MHC-DQA Haplotype 1',
            'Mamu-DQA Haplotype 2': 'MHC-DQA Haplotype 2',
            'Mamu-DQB Haplotype 1': 'MHC-DQB Haplotype 1',
            'Mamu-DQB Haplotype 2': 'MHC-DQB Haplotype 2',
            'Mamu-DPA Haplotype 1': 'MHC-DPA Haplotype 1',
            'Mamu-DPA Haplotype 2': 'MHC-DPA Haplotype 2',
            'Mamu-DPB Haplotype 1': 'MHC-DPB Haplotype 1',
            'Mamu-DPB Haplotype 2': 'MHC-DPB Haplotype 2',
            'Mamu-I Haplotype 1': 'MHC-I Haplotype 1',
            'Mamu-I Haplotype 2': 'MHC-I Haplotype 2',
        }, strict=False
        ).select(
            # Select only relevant columns
            'Animal ID', cs.starts_with('Batch'), 'GS ID',
            cs.starts_with('MHC-') & cs.matches(r'.*Haplotype [12]$'),
        ).drop_nulls(
            # Drop rows with missing Animal ID
            "Animal ID"
        ).filter(
            # Remove rows without any haplotypes
            # TODO: Only works when all input files have all these columns. Need to generalize
            ~(pl.col('MHC-A Haplotype 1').is_null()
              & pl.col('MHC-A Haplotype 2').is_null()
              & pl.col('MHC-B Haplotype 1').is_null()
              & pl.col('MHC-B Haplotype 2').is_null()
              & pl.col('MHC-DRB Haplotype 1').is_null()
              & pl.col('MHC-DRB Haplotype 2').is_null()
              & pl.col('MHC-DQA Haplotype 1').is_null()
              & pl.col('MHC-DQB Haplotype 2').is_null()
              & pl.col('MHC-DPA Haplotype 1').is_null()
              & pl.col('MHC-DPB Haplotype 2').is_null()
            #   & pl.col('MHC-I Haplotype 1').is_null()
            #   & pl.col('MHC-I Haplotype 2').is_null()
            )
        )
    
# .filter(
#         # Remove rows that aren't animals
#         #~pl.col("Animal ID").is_in(["Totals", "Ave"])
#         ~pl.col("GS ID(s)").is_in(["Totals", "Ave"])
#     )

# dfs = []
# for batch, file in zip(input_files["batch"].to_list(), input_files["file"].to_list()):
#     try:
#         dfs.append(parse_excel(file).with_columns(
#             Batch = pl.lit(batch
#         )))
#     except:
#         print(f"File {file} is not in correct format.")

# Read the batch mapping file
batches = pl.read_csv(batch_map, separator='\t', comment_prefix="#")

dfs = []
for batch, file in zip(input_files["batch"].to_list(), input_files["file"].to_list()):
    df = parse_excel(file)
    # Check if already has "Batch" column
    if "Batch" in df.columns:
        # Update the batch names
        df = df.with_columns(
            pl.col("Batch").replace(old=batches["Old"], new=batches["New"]).alias("Batch")
        )
    else:
        # Add the batch column
        df = df.with_columns(
            Batch = pl.lit(batch)
        )
    try:
        dfs.append(df)
    except:
        print(f"File {file} is not in correct format.")    

In [ ]:
# A key to convert 2017 haplotype names to 2021 haplotype names
haplotype_key = pl.read_csv(prefix + 'haplotype_key.tsv', separator='\t')
mapping = dict(
    zip(
        list(haplotype_key['2017 Haplotype']), list(haplotype_key['2021 Haplotype'])
    )
)

# Not always as simple as this
# mappings={
#     'a': '.01',
#     'b': '.02',
#     'c': '.03',
#     'd': '.04',
#     'e': '.05',
#     'f': '.06',
#     'g': '.07',
# }

nested = pl.concat(dfs, how='diagonal_relaxed').with_columns(
    # Update to 2021 nomenclature of abbreviated haplotypes
    # TODO: Deal with the abnormal haplotype values like those with "{}", "rec", etc.
    pl.col("MHC-A Haplotype 1").replace(mapping),
    pl.col("MHC-A Haplotype 2").replace(mapping),
    pl.col("MHC-B Haplotype 1").replace(mapping),
    pl.col("MHC-B Haplotype 2").replace(mapping),
).with_columns(
    # Combine the two haplotypes for each gene into arrays
    # TODO: Generalize this to work for any gene
    pl.concat_arr(['MHC-A Haplotype 1', 'MHC-A Haplotype 2']).alias('MHC-A'),
    pl.concat_arr(['MHC-B Haplotype 1', 'MHC-B Haplotype 2']).alias('MHC-B'),
    pl.concat_arr(['MHC-DRB Haplotype 1', 'MHC-DRB Haplotype 2']).alias('MHC-DRB'),
    pl.concat_arr(['MHC-DQA Haplotype 1', 'MHC-DQA Haplotype 2']).alias('MHC-DQA'),
    pl.concat_arr(['MHC-DQB Haplotype 1', 'MHC-DQB Haplotype 2']).alias('MHC-DQB'),
    pl.concat_arr(['MHC-DPA Haplotype 1', 'MHC-DPA Haplotype 2']).alias('MHC-DPA'),
    pl.concat_arr(['MHC-DPB Haplotype 1', 'MHC-DPB Haplotype 2']).alias('MHC-DPB'),
).drop(
    # Remove the split haplotype columns
    cs.ends_with('Haplotype 1') | cs.ends_with('Haplotype 2')
).rename(
    # Rename Id column to match demographics file
    {'Animal ID': 'Indiv'}
)

# # Join all parsed data with pedigree information
# nested = pl.read_csv(
#     # Add pedigree information
#     demographics,
#     separator='\t', comment_prefix="#", columns=["Id", "Sire", "Dam"], schema_overrides={
#         "Id": pl.String,
#         "Sire": pl.String,
#         "Dam": pl.String,
#     }
# ).join(
#     # Concatenate the parsed excel files
#     pl.concat(dfs, how='diagonal_relaxed'),
#     left_on="Id", right_on="Animal ID", how="left"

nested

In [ ]:
regex_unk = r'unk|Unk| |[(?]'

unpivoted = nested.unpivot(
    # Unpivot the MHC haplotype columns into rows
    cs.starts_with('MHC-'),
    index=['Indiv', 'Batch', 'GS ID'],
    variable_name='Gene',
    value_name='Diplotype'
).with_columns(
    # Set previously inferred to null (will be inferred later),
    # as well as any containing "?" or "unk" or "Unk"
    pl.col("Diplotype").arr.eval(
        pl.when(pl.element().str.contains(regex_unk)).then(
            None
        ).otherwise(
            pl.element()
        )
    ).alias("Diplotype")
).with_columns(
    pl.col("Diplotype").arr.eval(
        pl.when(pl.element().str.contains(regex_unk)).then(
            None #pl.lit("Inferred")
        ).when(pl.element().is_not_null()).then(
            pl.lit("Empirical"))
    ).alias("Haplotype Method")
)

unpivoted

In [ ]:
# Replace certain haplotypes with a table of manual corrections
modifications = pl.read_csv(modifications_file, separator='\t', comment_prefix="#", schema_overrides={
    'Indiv': pl.String,
}).with_columns(
    # Convert diplotypes into lists
    pl.col("Old Diplotype").str.split(",").list.to_array(2),
    pl.col("New Diplotype").str.split(",").list.to_array(2),
    pl.lit(["Modified", "Modified"]).alias("Haplotype Method").list.to_array(2),
    pl.lit("Modified").alias("Batch"),
    pl.lit(None).alias("GS ID"),
).rename({
    'New Diplotype': 'Diplotype'
}).select("Indiv", "Batch", 'GS ID', "Gene", "Diplotype", "Haplotype Method")

#modifications


# Old approach which duplicated rows
# modified = pl.concat([
#     # Keep rows that don't need modification
#     unpivoted.join(modifications, on=['Indiv', 'Gene'], how='anti'),
#     modifications
# ])

# Apply modifications
modified = unpivoted.join(modifications, on=['Indiv', 'Gene'], how='left').with_columns(
    # Use modified diplotype if exists, otherwise keep original
    pl.when(pl.col("Diplotype_right").is_null()).then(
        pl.col("Diplotype")
    ).when(pl.col("Diplotype").is_not_null()).then(
        pl.col("Diplotype_right")
    ).alias("Diplotype"),
    # TODO: Fix Haplotype Method since it applies to both haplotypes and not just one
    pl.when(pl.col("Haplotype Method_right").is_null()).then(
        pl.col("Haplotype Method")
    ).when(pl.col("Haplotype Method_right").is_not_null()).then(
        pl.col("Haplotype Method_right")
    ).alias("Haplotype Method")
).drop(cs.ends_with("_right"))

# Join all parsed data with pedigree information
modified = pl.read_csv(
    # Add pedigree information
    demographics,
    separator='\t', comment_prefix="#", columns=["Id", "Sire", "Dam"], schema_overrides={
        "Id": pl.String,
        "Sire": pl.String,
        "Dam": pl.String,
    }
).join(
    # Concatenate the parsed excel files
    modified,
    left_on="Id", right_on="Indiv", how="left"
)
# .rename({
#     # TODO: Remove this here and in later parts
#     'Diplotype': 'Haplotype'
# })

modified

In [ ]:
# Earlier attempt to keep empirical haplotypes over inferred, but didn't work as intended
# comparison = unpivoted.group_by("Id", "Gene", "Haplotype", "Haplotype Method"
# ).agg(
#     pl.len(), pl.first("Sire", "Dam"), "Batch",
    # Since there can only be one "Inferred" (at least with how this has been handled so far)
    # for a combination of Animal and Gene, we can just filter for "Empirical" to remove
    # any "Inferred" entries that should also then have "Empricial" entries
    # pl.when(pl.len() > 1)
    # .then(pl.col("Haplotype").filter(pl.col("Haplotype Method") == "Empirical"))
    # .otherwise("Haplotype")
    #pl.col("Haplotype").filter(pl.col("Haplotype Method") == "Empirical")
#).filter(pl.col("len") > 1 # TEST
#)

# Split empirical and inferred into separate tables
# TODO: Take into account when there are multiple empirical haplotypes for a single animal and gene.
# Using Debbie's compilation, there shouldn't be such cases, but there will be when I start processing
# each of the old batches in their original form
# empirical = unpivoted.filter(
#     pl.col("Haplotype Method") == "Empirical"
# )
# inferred = unpivoted.filter(
#     pl.col("Haplotype Method") == "Inferred"
# )

not_nulls = modified.filter(
    ~(pl.col("Haplotype Method") == [None, None])
) 

nulls = modified.filter(
    pl.col("Haplotype Method") == [None, None]
) 

# Keep only nulls that are not also empirical and then concatenate with empirical table
concatenated = pl.concat([not_nulls, nulls.join(not_nulls, on=['Id', 'Gene'], how='anti')])

concatenated

In [ ]:
# # TODO: Decide which to keep when there are multiple haplotypes recorded for same animal and gene
# concatenated.group_by("Id", "Gene").agg(pl.first("Sire", "Dam"), "Batch", "Haplotype", "Haplotype Method").filter(
#     pl.col("Haplotype").list.len() > 1
# )

In [ ]:
# Join with sire and dam haplotypes
trios = concatenated.join(
    # Join sire haplotypes
    concatenated.select('Id', 'Gene', 'Diplotype'),
        left_on=['Sire', 'Gene'], right_on=['Id', 'Gene'], how='left', suffix='_sire'
).join(
    # Join dam haplotypes
    concatenated.select('Id', 'Gene', 'Diplotype'),
        left_on=['Dam', 'Gene'], right_on=['Id', 'Gene'], how='left', suffix='_dam'
).with_columns(
    # Converts nulls to [null, null]
    # TODO: Is there a way to set as array initially instead of coverting after?
    pl.col("Diplotype_sire").fill_null([None, None]).cast(pl.Array(pl.String, 2)),
    pl.col("Diplotype_dam").fill_null([None, None]).cast(pl.Array(pl.String, 2)),
)

trios

In [ ]:
# TODO: Impute parents from offspring
# TODO: Allow storing both Empirical and Inferred for the same haplotype
# to be able to show extra confidence in the typing

# Infer offspring haplotype when parent is homozygous
inferred = trios.with_columns(
    # Find alleles homozygous in parents
    pl.when(pl.col("Diplotype_sire").arr.get(0) == pl.col("Diplotype_sire").arr.get(1)).then(
        pl.col("Diplotype_sire").arr.get(0))
    .alias("Paternal Inference"),
    pl.when(pl.col("Diplotype_dam").arr.get(0) == pl.col("Diplotype_dam").arr.get(1)).then(
        pl.col("Diplotype_dam").arr.get(0))
    .alias("Maternal Inference"),
    # Identify whether each parent is compatible with offspring
    # TODO: Take into account where paternity and maternity might separately be True but not together
    pl.when((pl.col("Diplotype_sire").arr.get(0).is_in(pl.col("Diplotype"))
     | pl.col("Diplotype_sire").arr.get(1).is_in(pl.col("Diplotype"))
    )).then(
        True
    ).when(pl.col("Diplotype").arr.contains(None) | (pl.col("Diplotype_sire") == [None, None])
    ).then(
        None
    ).otherwise(
        False
    ).alias("Haplotypic Paternity"),
    pl.when((pl.col("Diplotype_dam").arr.get(0).is_in(pl.col("Diplotype"))
     | pl.col("Diplotype_dam").arr.get(1).is_in(pl.col("Diplotype"))
    )).then(
        True
    ).when(pl.col("Diplotype").arr.contains(None) | (pl.col("Diplotype_dam") == [None, None])
    ).then(
        None
    ).otherwise(
        False
    ).alias("Haplotypic Maternity")
).with_columns(
    # Create the inferred haplotype
    pl.concat_arr([
        pl.col("Paternal Inference"),
        pl.col("Maternal Inference"),
    ]).alias("Inferred Haplotype")
).drop("Paternal Inference", "Maternal Inference"
).with_columns(
    # Combine existing empirical haplotypes with the inferred.
    # This is done using set logic with Empirical + (Emprirical - Inferred).
    # So, empirical haplotypes are given priority while also not duplicating them with inferred if they
    # have the same haplotypes. Homozygous inferred remove the second copy but are added back in in the next `with_columns(...)`
    pl.concat_list([pl.col("Diplotype").arr.to_list(), pl.col("Inferred Haplotype").arr.to_list().list.set_difference(
        pl.col("Diplotype").arr.to_list()
    )]).list.drop_nulls(
    ).alias("Empirical+Inferred")
).with_columns(
    # Add back in duplicate haplotypes when inferred haplotypes are homozygous that were removed when treated as sets
    pl.when(pl.col("Inferred Haplotype").arr.get(0) == pl.col("Inferred Haplotype").arr.get(1)).then(
        pl.col("Empirical+Inferred").list.concat(pl.col("Inferred Haplotype").arr.get(0))
    ).otherwise(
        pl.col("Empirical+Inferred")
    ).alias("Empirical+Inferred")
).with_columns(
    # Remove extra haplotypes, so that only two are kept.
    # Can become less than two though if some one or both are missing
    pl.col("Empirical+Inferred").list.slice(0, 2).list.sort()
).drop("Inferred Haplotype"
).with_columns(
    # Add back in nulls when necessary to make two haplotypes
    pl.when(pl.col("Empirical+Inferred").list.len() == 0
    ).then(
        pl.lit([None, None])
    ).when(pl.col("Empirical+Inferred").list.len() == 1
    ).then(
        pl.col("Empirical+Inferred").list.concat(pl.lit([None]))
    ).otherwise(
        pl.col("Empirical+Inferred")
    ).alias("Empirical+Inferred")
).with_columns(
    # Determine which of the haplotypes are empirical and which are inferred
    # by comparing whether the haplotype was one of the original haplotypes.
    # The next `with_columns` will correct an exception
    pl.when(pl.col("Empirical+Inferred").list.get(0).is_in(pl.col("Diplotype"))
    ).then(
        pl.lit("Empirical")
    ).when(~pl.col("Empirical+Inferred").list.get(0).is_in(pl.col("Diplotype"))
    ).then(
        pl.lit("Inferred")
    ).alias("Haplotype Status 1"),
    pl.when(pl.col("Empirical+Inferred").list.get(1).is_in(pl.col("Diplotype"))
    ).then(
        pl.lit("Empirical")
    ).when(~pl.col("Empirical+Inferred").list.get(1).is_in(pl.col("Diplotype"))
    ).then(
        pl.lit("Inferred")
    ).alias("Haplotype Status 2"),
).with_columns(
    # Change one of two "Empirical"s back to "Inferred" if the new haplotype is homozygous
    # while the old is heterozygous
    pl.when(
        # When the new haplotypes are homozygous, but the original are not
        (pl.col("Empirical+Inferred").list.get(0) == pl.col("Empirical+Inferred").list.get(1))
        & (pl.col("Diplotype").arr.get(0) != pl.col("Diplotype").arr.get(1))
    ).then(
        # Change the "Empricial" to "Inferred"
        pl.lit("Inferred")
    ).otherwise(
        pl.col("Haplotype Status 2")
    ).alias("Haplotype Status 2")
).with_columns(
    pl.concat_list(["Haplotype Status 1", "Haplotype Status 2"]).alias("Haplotype Status")
).drop("Haplotype Status 1", "Haplotype Status 2"
).with_columns(
    # Overwrite columns before inferrence
    pl.col("Empirical+Inferred").alias("Diplotype"),
    pl.col("Haplotype Status").alias("Haplotype Method")
).drop("Empirical+Inferred", "Haplotype Status")

# Unfortunately, this doesn't work since a named column can't be used inside `eval`
# .with_columns(
#     pl.col("Haplotype").arr.eval(
#         pl.element() == pl.col("Paternal Inference")
#     ).alias("Paternal Mendelian Error")
# )

inferred.filter(
    # Remove unknowns
    pl.col("Diplotype") != [None, None]
)#.write_excel("/master/abagwell/workspace/MHC/output/haplotypes_unpivoted.xlsx")

# # Shows which trios have Mendelian errors
# inferred.filter(
#     (pl.col("Haplotypic Paternity") == False) | (pl.col("Haplotypic Maternity") == False)
# ).filter(
#     pl.col("Gene").is_in(["MHC-A", "MHC-B"])
# ).sort("Id", "Gene"
# )


In [ ]:
# For checking trios
individual = inferred.filter(pl.col("Dam") == "33932"
).filter(
    pl.col("Gene") == "MHC-A"
).filter(
    pl.col("Diplotype") != [None, None]
).filter(
    pl.col("Haplotype Method") == ["Empirical", "Empirical"]
)
individual

In [ ]:
# Read kinship
# kinship = pl.read_csv("/master/abagwell/variant-analysis/results/rhesus/kinship/KING/WES3+WGS3_left_join.founders24.SNP.autosomal.kin",
# separator="\t", columns=["ID1", "ID2", "Kinship", "Error"]).with_columns(
#     # Find animal ID from longer sample name
#     pl.col("ID1").str.split("_").list.get(0).str.slice(3),
#     pl.col("ID2").str.split("_").list.get(0).str.slice(3),
# )

# Shows which trios have Mendelian errors and compare to kinship
inconsistencies = inferred.filter(
    (pl.col("Haplotypic Paternity") == False) | (pl.col("Haplotypic Maternity") == False)
).filter(
    pl.col("Gene").is_in(["MHC-A", "MHC-B"])
).sort("Id", "Gene"
)
# ).with_columns(
#     # Set IDs for comparison to kinship
#     pl.when((pl.col("Haplotypic Paternity") == False) & pl.col("Haplotypic Maternity") == False
#     ).then(
#         pl.col("Id")  # TODO: Not sure how to write if both parents don't appear correct
#     ).when(pl.col("Haplotypic Paternity") == False
#     ).then(
#        pl.col("Sire")
#     ).when(pl.col("Haplotypic Maternity") == False
#     ).then(
#         pl.col("Dam")
#     ).alias("ID2")
# ).join(
#     # Compare to kinship
#     kinship, left_on=["Id","ID2"], right_on=["ID1", "ID2"]
# )

In [ ]:
specific = inconsistencies.filter(
    #pl.col("Batch").is_in(["SNPRC22", "SNPRC23"])
)#.group_by("Id").agg(pl.len())
specific#.write_excel('/master/abagwell/workspace/MHC/output/inconsistent_haplotypes.SNPRC22-SNPRC23.xlsx')
#specific.write_excel('/master/abagwell/workspace/MHC/output/inconsistent_haplotypes.all.xlsx')

In [ ]:
# Find number of inferred vs empirical
inferred.filter(
    pl.col("Gene").is_in(["MHC-A", "MHC-B"])
).select("Haplotype Method").explode("Haplotype Method").group_by("Haplotype Method").agg(pl.len())

In [ ]:
# All unique haplotypes.
# Can be used to see if there are any abnormally named haplotypes that have not beeen accounted for
inferred.select("Diplotype").explode("Diplotype").rename({
    'Diplotype': 'Haplotype'
}).unique().sort("Haplotype")


In [ ]:
# Write to file
inferred.select(
    "Id", "Gene", "Diplotype", "Haplotype Method" # "Trio Inconsistency"
).filter(
    # Remove unknowns
    pl.col("Diplotype") != [None, None]
).explode(
    # Necessary to explode lists to be able to write as a TSV
    "Diplotype", "Haplotype Method"
# ).drop_nulls(
#     # Remove rows without a known haplotype
#     "Haplotype"
).rename({
    'Diplotype': 'Haplotype'
}).sort("Id", "Gene", "Haplotype"
).with_columns(
    # Add back parentheses when inferred
    pl.when(
        pl.col("Haplotype Method") == "Inferred"
    ).then(
        pl.lit("(") + pl.col("Haplotype") + pl.lit(")")
    ).otherwise(
        pl.col("Haplotype")
    ).alias("Haplotype")
).drop("Haplotype Method"
)#.filter(pl.col("Haplotype").str.contains("/")).select("Haplotype").unique()
#.write_csv("/master/abagwell/workspace/MHC/output/haplotypes_unpivoted.diplotype_exploded.tsv", separator="\t")

In [ ]:
# Repivot data. This is not for later steps
repivoted = inferred.explode(
    # Get only one haplotype per record
    "Diplotype", "Haplotype Method"
).rename({
    'Diplotype': 'Haplotype'
}).with_columns(
    # Add back parentheses when inferred
    pl.when(
        pl.col("Haplotype Method") == "Inferred"
    ).then(
        pl.lit("(") + pl.col("Haplotype") + pl.lit(")")
    ).otherwise(
        pl.col("Haplotype")
    ).alias("Haplotype")
).group_by(
    "Id", "Batch", "Gene"
).agg(
    # Recreate the diplotype field
    "Haplotype", pl.first("GS ID")
).rename({
    'Haplotype': 'Diplotype'
}).filter(
    # Drop nulls diplotypes
    pl.col("Diplotype") != [None, None]
).pivot(
    # Pivot back to having each gene as a separate column, but this time with inferences
    "Gene", index= ["Id", "Batch", "GS ID"], values="Diplotype"
).select(
    # Sort columns
    "Id", "Batch", "GS ID", "MHC-A", "MHC-B", "MHC-DPA", "MHC-DPB", "MHC-DQA", "MHC-DQB", "MHC-DRB",
).sort("Id")

repivoted#.write_excel("/master/abagwell/workspace/MHC/output/haplotypes_pivoted.xlsx")

In [ ]:
# Table of monitored alleles and their corresponding genes
monitored_alleles = pl.DataFrame(
    {
        "Gene": ["MHC-A", "MHC-B", "MHC-B", "MHC-B"],
        "Monitored Allele": ["A001", "B003", "B008", "B017"],
    }
)

# Create status columns for the presence or absence of each monitored allele
# using the existing table nomenclature. However, the previous table contains
# more information and should become the new standard
# Note: If such columns continue to be used in the future, names such as A1*001
# should be used over A001.
status = inferred.join(
    # Split into separate rows for each monitored allele
    monitored_alleles, on="Gene"
).explode("Diplotype", "Haplotype Method"
).rename({
    'Diplotype': 'Haplotype'
}).with_columns(
    # Find monitored allele status is regards to each single haplotype
    pl.when(pl.col("Haplotype").str.contains(pl.col("Monitored Allele"))).then(
        pl.lit(True)
    ).when(~pl.col("Haplotype").str.contains(pl.col("Monitored Allele"))).then(
        pl.lit(False)
    ).alias("Partial Allele Status")
).group_by(
    # Recombine
    "Id", "Gene", "Monitored Allele"
).agg(
    pl.first("Sire", "Dam"), "Haplotype", "Haplotype Method", "Partial Allele Status"
).rename({
    'Haplotype': 'Diplotype'
}).with_columns(
    pl.when(
        # Only one empirically true is needed to be "POSITIVE"
        ((pl.col("Haplotype Method").list.get(0) == "Empirical") & (pl.col("Partial Allele Status").list.get(0) == True))
        | ((pl.col("Haplotype Method").list.get(1) == "Empirical") & (pl.col("Partial Allele Status").list.get(1) == True))
    ).then(
        pl.lit("POSITIVE")
    ).when(
        # Otherwise, only one inferred true is needed to be "(POSITIVE)"
        ((pl.col("Haplotype Method").list.get(0) == "Inferred") & (pl.col("Partial Allele Status").list.get(0) == True))
        | ((pl.col("Haplotype Method").list.get(1) == "Inferred") & (pl.col("Partial Allele Status").list.get(1) == True))
    ).then(
        pl.lit("(POSITIVE)")
    ).when(
        # Otherwise, both must be empirically false to be "NEGATIVE"
        ((pl.col("Haplotype Method").list.get(0) == "Empirical") & (pl.col("Partial Allele Status").list.get(0) == False))
        & ((pl.col("Haplotype Method").list.get(1) == "Empirical") & (pl.col("Partial Allele Status").list.get(1) == False))
    ).then(
        pl.lit("NEGATIVE")
    ).when(
        # Otherwise, both must be inferred false to be "(NEGATIVE)"
        ((pl.col("Haplotype Method").list.get(0) == "Inferred") & (pl.col("Partial Allele Status").list.get(0) == False))
        & ((pl.col("Haplotype Method").list.get(1) == "Inferred") & (pl.col("Partial Allele Status").list.get(1) == False))
    ).then(
        pl.lit("(NEGATIVE)")
    ).alias("Allele Status")
)

status

In [ ]:
# Specific haplotypes have been inferred previously. But here, statuses are inferred.
# That is, when all parent haplotypes are known and it is then known that the offspring
# does not have one of the monitored alleles

prev_height = 0
cur_height = status.drop_nulls("Allele Status").height

# Each iteration is able to utilize inferred status from the previous iteration
while cur_height > prev_height:
    prev_height = cur_height
    status = status.join(
        # Join sire haplotypes
        status.select('Id', 'Gene', 'Monitored Allele', 'Allele Status'),
            left_on=['Sire', 'Gene', 'Monitored Allele'], right_on=['Id', 'Gene', 'Monitored Allele'], how='left', suffix='_sire'
    ).join(
        # Join dam haplotypes
        status.select('Id', 'Gene', 'Monitored Allele', 'Allele Status'),
            left_on=['Dam', 'Gene', 'Monitored Allele'], right_on=['Id', 'Gene', 'Monitored Allele'], how='left', suffix='_dam'
    ).with_columns(
        # Infer monitored allele status
        # Any definite positives should have already been inferred,
        # so this will serve to infer negatives
        pl.when(
            pl.col("Allele Status").is_not_null()
        ).then(
            pl.col("Allele Status")
        ).otherwise(
            pl.when(
                ((pl.col("Allele Status_sire") == "NEGATIVE") | (pl.col("Allele Status_sire") == "(NEGATIVE)"))
                & ((pl.col("Allele Status_dam") == "NEGATIVE") | (pl.col("Allele Status_dam") == "(NEGATIVE)"))
            ).then(
                pl.lit("(NEGATIVE)")
            )
        ).alias("Allele Status")
    ).drop("Allele Status_sire", "Allele Status_dam")

    cur_height = status.drop_nulls("Allele Status").height

In [ ]:
# Make one column for each monitored allele
# This uses the column names that are currently used elsewhere
final_status = status.pivot(
    # Create columns for each monitored allele
    "Monitored Allele",
    values="Allele Status",
    #aggregate_function="first",
    index=["Id", "Sire", "Dam"]
).rename({
    "A001": "A001 Status",
    "B003": "B003 Status",
    "B008": "B008 Status",
    "B017": "B017 Status",}
).sort("Id").select(
    # Sort statuses
    "Id", "A001 Status", "B003 Status", "B008 Status", "B017 Status"
).filter(
    # Remove individuals that are null for all monitored alleles
    ~(pl.col("A001 Status").is_null() & pl.col("B003 Status").is_null() & pl.col("B008 Status").is_null() & pl.col("B017 Status").is_null())
)

final_status#.write_csv("/master/abagwell/workspace/MHC/output/allele_statuses.tsv", separator="\t")

In [ ]:
# Convert to format that database currently uses
for_TAC = repivoted.join(
    # Merge the two dataframe
    final_status, on="Id"
).with_columns(
    # A column that the database has, but that I'm going to ignore
    pl.col('MHC-A').list.get(0).alias('Mamu-A Haplotype 1'),
    pl.col('MHC-A').list.get(1).alias('Mamu-A Haplotype 2'),
    pl.col('MHC-B').list.get(0).alias('Mamu-B Haplotype 1'),
    pl.col('MHC-B').list.get(1).alias('Mamu-B Haplotype 2'),
    pl.col('MHC-DPA').list.get(0).alias('Mamu-DPA Haplotype 1'),
    pl.col('MHC-DPA').list.get(1).alias('Mamu-DPA Haplotype 2'),
    pl.col('MHC-DPB').list.get(0).alias('Mamu-DPB Haplotype 1'),
    pl.col('MHC-DPB').list.get(1).alias('Mamu-DPB Haplotype 2'),
    pl.col('MHC-DQA').list.get(0).alias('Mamu-DQA Haplotype 1'),
    pl.col('MHC-DQA').list.get(1).alias('Mamu-DQA Haplotype 2'),
    pl.col('MHC-DQB').list.get(0).alias('Mamu-DQB Haplotype 1'),
    pl.col('MHC-DQB').list.get(1).alias('Mamu-DQB Haplotype 2'),
    pl.col('MHC-DRB').list.get(0).alias('Mamu-DRB Haplotype 1'),
    pl.col('MHC-DRB').list.get(1).alias('Mamu-DRB Haplotype 2'),
).rename({
    'Id': 'ANIMAL ID',
    'Batch': 'File Source',
    'GS ID': 'OC ID',  # TODO: Need to correct the abnormally named GS IDs
}).select(
    "ANIMAL ID", "File Source", "OC ID",
    "Mamu-A Haplotype 1", "Mamu-A Haplotype 2",
    "Mamu-B Haplotype 1", "Mamu-B Haplotype 2",
    "A001 Status", "B003 Status", "B008 Status", "B017 Status",
    "Mamu-DPA Haplotype 1", "Mamu-DPA Haplotype 2",
    "Mamu-DPB Haplotype 1", "Mamu-DPB Haplotype 2",
    "Mamu-DQA Haplotype 1", "Mamu-DQA Haplotype 2",
    "Mamu-DQB Haplotype 1", "Mamu-DQB Haplotype 2",
    "Mamu-DRB Haplotype 1", "Mamu-DRB Haplotype 2",
)

for_TAC.write_csv("/master/abagwell/workspace/MHC/output/MHC_for_TAC.tsv", separator="\t")